# 📘 Understanding Token Probabilities and Model Parameters

**Token probabilities** are fundamental to how language models generate text. This notebook explores how models predict the next word and how we can control their behavior using various parameters.

## 🎯 What You'll Learn

1. Downloading and loading language models locally
2. Understanding model vocabulary and tokenization
3. Generating word embeddings
4. Analyzing token probabilities
5. Controlling output with temperature, top-k, and top-p
6. Using sampling strategies for text generation
7. Combining parameters for optimal results

---

## 📊 Project Overview

**Goal:** Understand and control language model text generation behavior.

**Process:**
1. 📥 Download and load language models
2. 🔢 Analyze token probabilities
3. ⚙️ Experiment with generation parameters
4. 📈 Visualize probability distributions
5. 🎲 Compare sampling strategies

---

## 🚀 Section 1: Setup and Model Download

In [ ]:
import os
import json

from huggingface_hub import snapshot_download
from transformers import AutoTokenizer, AutoModelForCausalLM

import torch
import torch.nn.functional as F

import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
MODEL_NAME = "Qwen/Qwen2.5-0.5B"
MODEL_DIR = "saved_model"

def download_model_locally(model_name: str, local_dir: str) -> None:
    # Create local directory if not exists
    os.makedirs(local_dir, exist_ok=True)

    # Download entire repository to the specified local directory
    snapshot_download(repo_id=model_name, cache_dir=local_dir, local_dir=local_dir)
    print(f"Model downloaded locally to dir '{local_dir}'")


download_model_locally(model_name = MODEL_NAME, local_dir = MODEL_DIR)

---

## 📚 Section 2: Model Vocabulary Overview

The model's vocabulary is a dictionary mapping words (tokens) to unique numerical IDs. These IDs are used internally by the model to process text.

In [ ]:
# load json as dict
def load_json(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data

tokenizer_dict = load_json(os.path.join(MODEL_DIR, "tokenizer.json"))

# getting sequence number of a tocken
print(tokenizer_dict["model"]["vocab"]["world"])
print(tokenizer_dict["model"]["vocab"]["old"])
print(tokenizer_dict["model"]["vocab"]["age"])
print(tokenizer_dict["model"]["vocab"]["face"])

In [ ]:
vocabulary_dict = load_json(os.path.join(MODEL_DIR, "vocab.json"))
print(vocabulary_dict["world"])
print(vocabulary_dict["old"])
print(vocabulary_dict["age"])
print(vocabulary_dict["face"])

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)
model = AutoModelForCausalLM.from_pretrained(MODEL_DIR)

In [ ]:
print(tokenizer.vocab['world'])
print(tokenizer.vocab['old'])
print(tokenizer.vocab['age'])
print(tokenizer.vocab['face'])

---

## 🔑 Section 3: Special Tokens

Sometimes model developers add special tokens for training purposes. Special tokens help identify the start and end of specific instructions and character sequences like:

- User input
- System instructions
- Images
- Videos
- Tool calls

### 💡 Important Note

For some small models, it is important to use these tokens during prompting as it can significantly improve the results.

In [ ]:
tokenizer.get_added_vocab()

---

## 🔢 Section 4: Tokenization

### 📝 The Encode Method

The tokenizer's  method converts text into token numbers from the vocabulary.

**Important:** Token number is **NOT THE SAME** as the embedding of the word. Token numbers are just indices in the vocabulary, while embeddings are dense vector representations.

In [ ]:
def get_token_numbers(text):
    tokens = tokenizer.encode(text, add_special_tokens=False)
    return tokens, len(tokens)

In [ ]:
# Example prompt
prompts = ["old", "cat", "old cat", "Old cat"]

# Get token numbers
for prompt in prompts:
  tokens, num_tokens = get_token_numbers(prompt)
  print(f"Prompt: {prompt}")
  print(f"Token IDs: {tokens}")
  print("-"*50)

### ❓ Question to Consider

Why does the word  have number **4616**, but in combination with  it changes to **8251**?

**Answer:** The tokenizer may combine frequent word pairs into single tokens for efficiency. Context matters in tokenization!

---

## 📈 Section 5: Token Probabilities

Understanding how the model assigns probabilities to potential next tokens is crucial for controlling text generation.

In [ ]:
def get_next_token_probabilities(prompt):

    # Tokenize input prompt
    inputs = tokenizer(prompt, return_tensors="pt")

    # Get model output logits
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    # Get logits for the last position (next token prediction)
    next_token_logits = logits[:, -1, :]

    # Convert logits to probabilities
    next_token_probs = F.softmax(next_token_logits, dim=-1)

    # Convert to dictionary of token -> probability
    probs_dict = {tokenizer.decode([token_id]).strip(): prob.item()
                  for token_id, prob in enumerate(next_token_probs[0])}

    return probs_dict

In [ ]:
prompt = "I live in Paris, this city is very"
probs_dict = get_next_token_probabilities(prompt)

# order dict desc by value
probs_dict = {k: v for k, v in sorted(probs_dict.items(), key=lambda item: item[1], reverse=True)}

#plot probabilities distribution
sns.barplot(x=list(probs_dict.values())[:20], y=list(probs_dict.keys())[:20])
plt.show()

---

## ⚙️ Section 6: Parameters for Controlling Token Probability Distribution

### 🌡️ Temperature

Controls how **random or predictable** the model's output is.

- **Low temperature** (e.g., 0.1) makes outputs more **deterministic and repetitive**
- **High temperature** (e.g., 1.5) increases **creativity and variety**, but also randomness

**Use Case:** Lower for factual text, higher for creative writing.

---

### 🔢 Top-k Sampling

Limits the next word choices to the **top k highest probability tokens**.

- Only the *k* most likely words are considered; others are ignored
- Shrinks the pool of possible next words, reducing nonsensical outputs

**Use Case:** Balances randomness with sensible options.

---

### 🎯 Top-p (Nucleus) Sampling

Limits next word choices to the **smallest set whose probabilities sum to p** (e.g., 0.9).

- The size of this set changes dynamically depending on word probabilities
- Keeps all words needed to cover most of the probability mass

**Use Case:** More adaptive than top-k, better balance of diversity.

---

### 🎲 Sampling

Instead of picking the single most likely next word, sampling **randomly chooses** according to the probability distribution after temperature, top-k, and top-p adjustments.

- Introduces diversity and creativity in outputs
- Without sampling, the model always picks the highest probability word (deterministic)

---

### 🚫 Repetition Penalty

Discourages the model from **repeating the same words or phrases** over and over.

- Reduces probabilities for already generated tokens
- Helps prevent loops and dull, repetitive text

**Use Case:** Important for longer generations to keep text engaging and coherent.

---

### 📝 Quick Reference

⚙️ **Temperature** controls boldness vs. safety  
📋 **Top-k** limits choices to a fixed shortlist  
🎯 **Top-p** chooses from a flexible shortlist by probability  
🎲 **Sampling** adds randomness for creativity  
🚫 **Repetition penalty** avoids boring repetition

In [ ]:
def get_probabilities_with_temperature(prompt, temperature):
    """
    Apply temperature scaling to logits to control randomness in token selection.
    Lower temperature = more deterministic, higher temperature = more random.
    """
    # Tokenize the input prompt into tensor format
    inputs = tokenizer(prompt, return_tensors="pt")

    # Disable gradient computation for inference
    with torch.no_grad():
        # Get model outputs (logits for all positions)
        outputs = model(**inputs)
        # Extract logits for the last position (next token prediction)
        logits = outputs.logits[:, -1, :]

    # Scale logits by temperature (higher temp = more uniform distribution)
    scaled_logits = logits / temperature

    # Convert scaled logits to probabilities using softmax
    probs = F.softmax(scaled_logits, dim=-1)

    # Return dictionary mapping token strings to probabilities
    return {tokenizer.decode([i]).strip(): prob.item() for i, prob in enumerate(probs[0])}

def get_probabilities_with_top_k(prompt, top_k):
    """
    Filter to only the top-k most probable tokens, setting others to zero probability.
    This reduces the vocabulary to only the k most likely next tokens.
    """
    # Tokenize the input prompt into tensor format
    inputs = tokenizer(prompt, return_tensors="pt")

    # Disable gradient computation for inference
    with torch.no_grad():
        # Get model outputs (logits for all positions)
        outputs = model(**inputs)
        # Extract logits for the last position (next token prediction)
        logits = outputs.logits[:, -1, :]

    # Apply top-k filtering if k > 0
    if top_k > 0:
        # Get the k-th largest value to use as threshold
        kth_largest_value = torch.topk(logits, top_k)[0][..., -1, None]
        # Create mask for tokens below the threshold
        indices_to_remove = logits < kth_largest_value
        # Set filtered tokens to negative infinity (will become 0 probability)
        logits = logits.masked_fill(indices_to_remove, float('-inf'))

    # Convert logits to probabilities using softmax
    probs = F.softmax(logits, dim=-1)

    # Only return tokens with non-zero probabilities (efficient filtering)
    non_zero_probs = {}
    for i, prob in enumerate(probs[0]):
        if prob.item() > 0:
            # Decode token ID to string and strip whitespace
            token_str = tokenizer.decode([i]).strip()
            non_zero_probs[token_str] = prob.item()

    return non_zero_probs

def get_probabilities_with_top_p(prompt, top_p):
    """
    Apply nucleus sampling (top-p) to select the smallest set of tokens
    whose cumulative probability is at least p.
    """
    # Tokenize the input prompt into tensor format
    inputs = tokenizer(prompt, return_tensors="pt")

    # Disable gradient computation for inference
    with torch.no_grad():
        # Get model outputs (logits for all positions)
        outputs = model(**inputs)
        # Extract logits for the last position (next token prediction)
        logits = outputs.logits[:, -1, :]

    # Sort logits in descending order to work with cumulative probabilities
    sorted_logits, sorted_indices = torch.sort(logits, descending=True)
    # Convert sorted logits to probabilities
    sorted_probs = F.softmax(sorted_logits, dim=-1)
    # Calculate cumulative probabilities (running sum)
    cumulative_probs = torch.cumsum(sorted_probs, dim=-1)

    # Find where cumulative probability exceeds top_p threshold
    sorted_indices_to_remove = cumulative_probs > top_p
    # Shift mask to include the first token that exceeds threshold
    sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
    # Keep the first token (index 0) to ensure we always have at least one token
    sorted_indices_to_remove[..., 0] = 0

    # Map back to original indices using the sorted indices
    indices_to_remove = sorted_indices[sorted_indices_to_remove]
    # Set filtered tokens to negative infinity (will become 0 probability)
    logits[:, indices_to_remove] = float('-inf')

    # Convert final logits to probabilities
    probs = F.softmax(logits, dim=-1)

    # Return dictionary mapping token strings to probabilities
    return {tokenizer.decode([i]).strip(): prob.item() for i, prob in enumerate(probs[0])}

---

## 🌡️ Section 7.1: Experimenting with Temperature

Let's visualize how temperature affects the probability distribution of next tokens.

In [ ]:
for temp in [0.1, 0.5, 5.0, 10.0]:
    print(f"Temperature {temp}")
    probs_dict = get_probabilities_with_temperature(prompt, temp)
    probs_dict = {k: v for k, v in sorted(probs_dict.items(), key=lambda item: item[1], reverse=True)}

    #plot probabilities distribution
    sns.barplot(x=list(probs_dict.values())[:20], y=list(probs_dict.keys())[:20])
    plt.show()

---

## 🔢 Section 7.2: Experimenting with Top-k Sampling

Observe how different k values affect which tokens are considered.

In [ ]:
for k in [1, 5, 10, 50]:
    print(f"Top K: {k}")
    probs_dict = get_probabilities_with_top_k(prompt, k)
    probs_dict = {k: v for k, v in sorted(probs_dict.items(), key=lambda item: item[1], reverse=True)}

    #plot probabilities distribution
    sns.barplot(x=list(probs_dict.values())[:20], y=list(probs_dict.keys())[:20])
    plt.show()

---

## 🎯 Section 7.3: Experimenting with Top-p (Nucleus) Sampling

See how different p values dynamically adjust the token selection pool.

In [ ]:
for p in [0.1, 0.5, 0.9, 1.0]:
    print(f"Top P: {p}")
    probs_dict = get_probabilities_with_top_p(prompt, p)
    probs_dict = {k: v for k, v in sorted(probs_dict.items(), key=lambda item: item[1], reverse=True)}

    #plot probabilities distribution
    sns.barplot(x=list(probs_dict.values())[:20], y=list(probs_dict.keys())[:20])
    plt.show()

---

## 🎲 Section 8: Greedy vs. Sampling

### 🎯 Greedy Decoding (No Sampling)

Without sampling, the model always picks the highest probability token at each step, leading to deterministic outputs.

In [ ]:
def generate_response(prompt, max_new_tokens=20):
    inputs = tokenizer(prompt, return_tensors="pt")
    output_ids = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id
        )
    response = tokenizer.decode(output_ids[0], skip_special_tokens=False)
    return response

In [ ]:
response = generate_response(prompt)
print(f"Generated response: {response}")

In [ ]:
def generate_response_use_sampling(prompt, max_new_tokens=20):
    inputs = tokenizer(prompt, return_tensors="pt")
    output_ids = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
        )
    response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return response

In [ ]:
response = generate_response_use_sampling(prompt)
print(f"Generated response: {response}")

In [ ]:
def generate_with_repetition_penalty(prompt, max_new_tokens=20, repetition_penalty=1.1):
    """
    Generate text using HuggingFace's built-in repetition penalty.

    Args:
        prompt: Input text prompt
        max_new_tokens: Maximum number of new tokens to generate
        repetition_penalty: Penalty factor (>1.0 reduces repetition, 1.0 = no penalty)
    """
    # Tokenize the input prompt
    inputs = tokenizer(prompt, return_tensors="pt")

    # Generate with repetition penalty using HuggingFace's built-in method
    output_ids = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        repetition_penalty=repetition_penalty,  # Built-in repetition penalty
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id,
    )

    # Decode and return the response
    response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return response

In [ ]:
test_prompt = "The story begins with a cat. The "

# Test different penalty values
penalties = [0.9, 1.8]

for penalty in penalties:
    print(f"Repetition Penalty: {penalty}")
    response = generate_with_repetition_penalty(test_prompt, max_new_tokens=15, repetition_penalty=penalty)
    print(f"Generated: {response}")
    print("-" * 50)

---

## 📚 Summary

### ✨ Key Concepts Covered

1. **Model Setup**: Downloading and loading language models locally
2. **Vocabulary**: Understanding how words map to token IDs
3. **Special Tokens**: Using special tokens for structured prompts
4. **Tokenization**: Converting text to numerical representations
5. **Embeddings**: Dense vector representations that capture semantic meaning
6. **Token Probabilities**: How models predict the next word
7. **Parameters**:

   7.1 **Temperature**: Controlling randomness in outputs

   7.2 **Top-k Sampling**: Limiting choices to top k probable tokens

   7.3 **Top-p Sampling**: Dynamic probability-based token selection
8. **Sampling Strategies**: Greedy vs. probabilistic decoding
9. **Repetition Penalty**: Preventing repetitive outputs

### 💡 Best Practices

- ✅ Use **lower temperature** for factual, deterministic outputs
- ✅ Use **higher temperature** for creative, varied outputs
- ✅ Combine **top-k** and **top-p** for balanced diversity
- ✅ Enable **sampling** for more natural, creative text
- ✅ Apply **repetition penalty** to avoid loops in longer generations
- ✅ Experiment with parameter combinations to find optimal settings

### 🎯 Next Steps

- 🔹 Experiment with different prompt formats
- 🔹 Try various parameter combinations
- 🔹 Explore how special tokens affect model behavior
- 🔹 Compare outputs across different temperature settings
- 🔹 Build applications that leverage these techniques

---

### 🎓 Congratulations!

You now understand how language models generate text and how to control their behavior using various parameters. Practice by experimenting with different settings and observe how they affect the generated outputs.
